
<br>
Observation des réseaux de relations entre les différentes occupations des écrivain-es.<br>


In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt

L'adresse de DBPedia, où la requête doit être adressée.

In [2]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

Le format du résultat de la requête. L'objet, en python, sera un dictionnaire.

In [3]:
sparql.setReturnFormat(JSON)

La requête, qui trouve les Writers et leurs occupations.

In [4]:
sparql.setQuery(
    "PREFIX dbo: <http://dbpedia.org/ontology/>\n"
    "PREFIX dbr: <http://dbpedia.org/resource/>\n"
    "\n"
    "SELECT DISTINCT ?person ?occupation\n"
    "WHERE {\n"
    "    ?person ?a dbr:Writer ;\n"
    "            dbo:occupation ?occupation .\n"
    "}\n"
)

Envoyer la requête.

In [5]:
results = sparql.queryAndConvert()

Aperçu de la structure de l'objet retourné. Le premier niveau du dictionnaire.

In [6]:
results.keys()

dict_keys(['head', 'results'])

L'entrée "head" (en-tête), qui décrit la structure des données.

In [7]:
results["head"]

{'link': [], 'vars': ['person', 'occupation']}

L'entrée "results".

In [8]:
for i, j in results["results"].items():
    if type(j) == list:
        print(f'results["results"]["{i}"]:', type(j), j[:2])
    elif type(j) == dict:
        print(
            f'results["results"]["{i}"]:', type(j), list(j.keys())[:3]
        )
    else:
        print(f'results["results"]["{i}"]:', type(j), j)

results["results"]["distinct"]: <class 'bool'> False
results["results"]["ordered"]: <class 'bool'> True
results["results"]["bindings"]: <class 'list'> [{'person': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Caiseal_Mór'}, 'occupation': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Musician'}}, {'person': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Caiseal_Mór'}, 'occupation': {'type': 'uri', 'value': 'http://dbpedia.org/resource/Artist'}}]


Seul ce qui se trouve dans ['results']['bindings'] m'intéresse. Je commence par construire un simple liste de tuples selon le schéma (person, occupation).

In [9]:
pairs = [
    (i["person"]["value"], i["occupation"]["value"])
    for i in results["results"]["bindings"]
]
pairs[:3]

[('http://dbpedia.org/resource/Caiseal_Mór',
  'http://dbpedia.org/resource/Musician'),
 ('http://dbpedia.org/resource/Caiseal_Mór',
  'http://dbpedia.org/resource/Artist'),
 ('http://dbpedia.org/resource/Caiseal_Mór',
  'http://dbpedia.org/resource/Writer')]

Pour augmenter le nombre de données (la raison apparaitra plus bas, mais repose sur la limitation à 10000 résultats par requête auprès de dbpedia), je fais également des requêtes avec d'autres occupations que Writers.

In [10]:
queries = {}
separated_results = {}
pairs_supp = {}
groups = ["Novelist", "Poet", "Playwright"]
for i in groups:
    queries[i] = SPARQLWrapper("http://dbpedia.org/sparql")
    queries[i].setReturnFormat(JSON)
    queries[i].setQuery(
        "PREFIX dbo: <http://dbpedia.org/ontology/>\n"
        "PREFIX dbr: <http://dbpedia.org/resource/>\n"
        "\n"
        "SELECT DISTINCT ?person ?occupation\n"
        "WHERE {\n"
        f"    ?person ?a dbr:{i} ;\n"
        "            dbo:occupation ?occupation .\n"
        "}\n"
    )
    separated_results[i] = sparql.queryAndConvert()
    pairs_supp[i] = [
        (i["person"]["value"], i["occupation"]["value"])
        for i in separated_results[i]["results"]["bindings"]
    ]
    pairs.extend(pairs_supp[i])

J'enlève le début des URI, pour ne garder que les noms et les occupations.

In [11]:
clean_pairs = [
    (
        i.replace("http://dbpedia.org/resource/", ""),
        j.replace("http://dbpedia.org/resource/", ""),
    )
    for i, j in pairs
]
clean_pairs[:3]

[('Caiseal_Mór', 'Musician'),
 ('Caiseal_Mór', 'Artist'),
 ('Caiseal_Mór', 'Writer')]

Certaines paires me sont inutiles, celles qui ont comme occupation "[nom de la personne]_PersonFunction".

In [12]:
[i for i in clean_pairs if "PersonFunction" in i[1]][:5]

[('Caitlin_Flanagan', 'Caitlin_Flanagan__PersonFunction__1'),
 ('Caleb_Garling', 'Caleb_Garling__PersonFunction__1'),
 ('Calla_Curman', 'Calla_Curman__PersonFunction__1'),
 ('Camille_Lemonnier', 'Camille_Lemonnier__PersonFunction__1'),
 ('Carl_Binder', 'Carl_Binder__PersonFunction__1')]

Je construis donc une nouvelle liste sans ces paires.

In [13]:
non_trivial_pairs = [
    i for i in clean_pairs if "PersonFunction" not in i[1]
]
len(non_trivial_pairs)

25692

Il y a des répétitions de paires, puisque les Novelist, les Poets, etc., sont souvent aussi catégorisé-es comme Writer. En fait, l'ajout des nouveaux groupes n'a peut-être pas été si utile. [À faire, peut-être: ajouter des occupations liées à la littérature mais plutôt, par exemple, du côté des éditeurices, des bibliothécaires, des critiques. Ou, à l'inverse, des artistes, des musicien-nes, pour voir si les activités annexes sont du même type, s'il y a des recoupements, etc.]

In [14]:
uniqpairs = list(set(non_trivial_pairs))
len(uniqpairs)

6423

Puisque ce sont les interactions entre occupations qui m'intéresse, je ne vais garder que les données qui concernent les personnes avec au moins deux entrées. Je commence par faire une liste des noms.

In [15]:
names = [i for i, j in uniqpairs]
names[:3]

['Jan_Coffey', 'Aleš_Šteger', 'Mark_Osborne_(filmmaker)']

In [16]:
names_repeated = [i for i in names if names.count(i) > 1]

Le nombre total d'occurences de noms.

In [17]:
len(names)

6423

Le nombre d'occurences de noms qui apparaissent plusieurs fois.

In [18]:
len(names_repeated)

5021

Le nombre de noms qui apparaissent plusieurs fois. (Les données sont plus minces que ce que je pouvais espérer et il faudrait probablement opter pour une autre manière d'interroger DBPedia.)

In [19]:
len(set(names_repeated))

1782

Les premiers noms.

In [20]:
uniqnames = list(set(names_repeated))
uniqnames.sort()
for i in uniqnames[:5]:
    print(i)

A._C._Sreehari
A._Dorian_Otvos
A._N._Murthy_Rao
Aaron_Ehasz
Abd_Al_Munim_Al_Gilyani


Puisque chaque nom est unique, je peux utiliser les noms comme clés pour un dictionnaire. La valeur qui y sera attribuée sera une liste des occupations.

In [21]:
d = {}
for i in uniqnames:
    d[i] = []

Observer le dictionnaire avant d'y mettre les données.

In [22]:
for i in list(d.items())[:3]:
    print(i)

('A._C._Sreehari', [])
('A._Dorian_Otvos', [])
('A._N._Murthy_Rao', [])


Iterate sur les éléments de la variable clean_pairs qui contient les paires person/occupation, et append la liste des occupations de la personne

In [23]:
for name, occupation in uniqpairs:
    if name in uniqnames:
        d[name].append(occupation)

Observer une partie du dictionnaire.

In [24]:
for i in list(d.items())[:3]:
    print(i)

('A._C._Sreehari', ['Poet', 'Teacher', 'Lyricist', 'Writer'])
('A._Dorian_Otvos', ['Composer', 'Writer'])
('A._N._Murthy_Rao', ['Professor', 'Writer'])


L'occupation "Writer" est très présente C'est pour cela que j'ai fait des requêtes supplémentaires. Cela a permis d'ajouter un certain nombre d'entrées.

In [25]:
len([i for i, j in list(d.items()) if "Writer" not in j])

600

Mais la question se pose, tout de même, de l'intérêt ici de l'occupation "Writer" (qui est aussi, dans DBPedia, une méta-occupation). Je préfère la retirer, étant donné que les poète-sses, les romancier-ères, les dramaturges, sont aussi des écrivain-es.

In [26]:
data_to_pop = [
    i for i in list(d.keys()) if len(d[i]) < 3 and "Writer" in d[i]
]
len(data_to_pop)

518

In [27]:
for i in data_to_pop:
    d.pop(i)

Le nombre d'entrées restantes dans le dictionnaires.

In [28]:
len(d)

1264

Il va désormais s'agir d'explorer les relations entre occupations en utilisant l'analyse de réseaux. Pour créer un réseau reliant les occupations les unes aux autres, il faut d'abord créer des relations entre occupations, des nouvelles paires occupation/occupation.

In [29]:
occ_pairs = []
for name, occupations in d.items():
    for occ_one in occupations:
        a = occupations.copy()
        a.remove(occ_one)
        for occ_two in a:
            p = (occ_one, occ_two)
            occ_pairs.append(p)

Certaines paires ont plusieurs occurences: quand deux occupations cohabitent chez plusieurs personnes.

In [30]:
[i for i in occ_pairs if 'Biographer' in i][:10]

[('Biographer', 'Author'),
 ('Biographer', 'Screenwriter'),
 ('Author', 'Biographer'),
 ('Screenwriter', 'Biographer'),
 ('Biographer', 'Novelist'),
 ('Biographer', 'Activism'),
 ('Novelist', 'Biographer'),
 ('Activism', 'Biographer'),
 ('Biographer', 'Writer'),
 ('Biographer', 'Screenwriter')]

Plutôt que d'avoir ainsi des doublons, on peut représenter le nombre d'occurences comme la force de la relation.

In [31]:
c = Counter(occ_pairs)
relations = [(i[0], i[1], j) for i, j in list(c.items())]
relations[:10]

[('Poet', 'Teacher', 6),
 ('Poet', 'Lyricist', 7),
 ('Poet', 'Writer', 118),
 ('Teacher', 'Poet', 6),
 ('Teacher', 'Lyricist', 1),
 ('Teacher', 'Writer', 15),
 ('Lyricist', 'Poet', 7),
 ('Lyricist', 'Teacher', 1),
 ('Lyricist', 'Writer', 9),
 ('Writer', 'Poet', 118)]

Créer un graphe avec ces relations.

In [32]:
G = nx.Graph()
for relation in relations:
    node1, node2, weight = relation
    G.add_edge(node1, node2, weight=weight)

Hélas, il est déjà minuit: délai pour poster ce carnet (que je continue donc sur un autre fichier pendant la suite de la nuit).